# Generate HTML reports with D3 graphs using Python, Plotly, and Pandas

You can download the example report that this script generate from this link.

<a href="d3.js">d3.js</a> is an amazing JavaScript library for creating interactive, online graphics and charts. Plotly lets you create d3.js charts using Python, R, or MATLAB. This IPython notebook shows you how to embed these charts in an HTML report that you can then share by email or <a href="">host on a website</a>.

Once you've created your report generation script, you can automate it with a task scheduler like <a href="http://moderndata.plot.ly/update-plotly-charts-with-cron-jobs-and-python/">cron</a> on a Ubuntu server, and even email it email it to yourself or your team with the Python <a href="https://docs.python.org/2/library/smtplib.html">smtplib</a> library.

In [1]:
import plotly as py
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import time as dt_tm
from datetime import date as dt_date

import plotly.plotly as py
import plotly.tools as plotly_tools
from plotly.graph_objs import *

import os
import tempfile
os.environ['MPLCONFIGDIR'] = tempfile.mkdtemp()
from matplotlib.finance import quotes_historical_yahoo
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde

from IPython.display import HTML

py.sign_in("DaiYu", "kgx7wxp3ro")

## Step 1: Generate 2 graphs and 2 tables for report with Plotly Python client

### First graph: 2014 Apple stock data with moving average

Let's grab Apple stock data using the <a href="http://matplotlib.org/api/finance_api.html">matplotlib finance</a> model from 2014, then take a moving average with a <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.convolve.html">numpy convolution</a>.

In [2]:
x = []
y = []
ma = []

def moving_average(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

date1 = dt_date( 2014, 1, 1 )
date2 = dt_date( 2014, 12, 12 )
quotes = quotes_historical_yahoo('AAPL', date1, date2)
if len(quotes) == 0:
    print("Couldn't connect to yahoo trading database")
else:
    dates = [q[0] for q in quotes]
    y = [q[1] for q in quotes]
    for date in dates:
        x.append(datetime.fromordinal(int(date))\
                .strftime('%Y-%m-%d')) # Plotly timestamp format
    ma = moving_average(y, 10)

C:\Anaconda3\lib\site-packages\matplotlib\finance.py:485: MatplotlibDeprecationWarning:

This function has been deprecated in 1.4 in favor of `quotes_historical_yahoo_ochl`, which maintains the original argument order, or `quotes_historical_yahoo_ohlc`, which uses the open-high-low-close order. This function will be removed in 1.5



Now graph the data with Plotly. See here for Plotly's <a href="https://plot.ly/python/line-and-scatter/">line plot syntax</a> and here for <a href="https://plot.ly/python/getting-started/">getting started</a> with the Plotly Python client.

In [3]:
xy_data = Scatter( x=x, y=y, mode='markers', marker=Marker(size=4), name='AAPL' )
# vvv clip first and last points of convolution
mov_avg = Scatter( x=x[5:-4], y=ma[5:-4], \
                  line=Line(width=2,color='red'), name='Moving average' )
data = Data([xy_data, mov_avg])

py.iplot(data, filename='apple stock moving average')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~DaiYu/0 or inside your plot.ly account where it is named 'apple stock moving average'


Save the plot URL - we'll use it when generating the report later.

In [4]:
first_plot_url = py.plot(data, filename='apple stock moving average', auto_open=False,)
print(first_plot_url)

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~DaiYu/0 or inside your plot.ly account where it is named 'apple stock moving average'
https://plot.ly/~DaiYu/0


### 2 summary tables - One using Pandas and one using HTML

First table: Use Pandas <a href="http://pandas.pydata.org/pandas-docs/version/0.15.1/generated/pandas.DataFrame.to_html.html">to_html()</a> routine to convert Pandas data frame to HTML table. Replace default table styling with <a href="http://getbootstrap.com/css">Bootstrap</a> table styling.

In [5]:
summary_table_1 = df.describe()
summary_table_1 = summary_table_1\
    .to_html()\
    .replace('<table border="1" class="dataframe">','<table class="table table-striped">') # use bootstrap styling

NameError: name 'df' is not defined

In [ ]:
html_string = '''
<html>
    <head>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
        <style>body{ margin:0 100; background:whitesmoke; }</style>
    </head>
    <body>
        <h1>2014 technology and CPG stock prices</h1>

        <!-- *** Section 1 *** --->
        <h2>Section 1: Apple Inc. (AAPL) stock in 2014</h2>
        <iframe width="1000" height="550" frameborder="0" seamless="seamless" scrolling="no" \
src="''' + first_plot_url + '''.embed?width=800&height=550"></iframe>
        <p>Apple stock price rose steadily through 2014.</p>
    </body>
</html>'''

Finally, write the html string to a local file.

In [ ]:
f = open('test.html','w')
f.write(html_string)
f.close()